In [11]:
%pip install --quiet -U langchain langgraph langchainhub langchain_openai tavily-python python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
from dotenv import load_dotenv

load_dotenv()


True

In [20]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

from langchain_openai import ChatOpenAI

from langgraph.prebuilt import ToolExecutor

tools = [TavilySearchResults(max_results=1)]

model = ChatOpenAI(temperature=0, streaming=True)

In [21]:
from langchain_core.utils.function_calling import convert_to_openai_function

functions = [convert_to_openai_function(tool) for tool in tools]
model = model.bind_functions(functions)

In [35]:
# from langchain.prompts import ChatPromptTemplate

# prompt = ChatPromptTemplate(
#     [
#         ("system", "You are a helpful AI assistant."),
#     ]

# )

prompt = hub.pull("hwchase17/openai-functions-agent")

tool_executor = ToolExecutor(tools)


In [31]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]


In [32]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]

    if "function_call" not in last_message.additional_kwargs:
        return "end"
    
    else:
        return "continue"

def call_model(state):
    messages = state["messages"]
    response = model.invoke(messages)

    return {"messages": [response]}

def call_tool(state):
    messages = state["messages"]
    last_message = messages[-1]

    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input = json.loads(last_message.additional_kwargs["function_call"]["arguments"])
    )

    response = tool_executor.invoke(action)

    function_message = FunctionMessage(content=str(response), name=action.tool)

    return {"messages": [function_message]}

def call_formatter(state):
    

In [33]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent", should_continue, {"continue": "action", "end": END}
)

workflow.add_edge("action", "agent")

app = workflow.compile()

In [38]:
from langchain_core.messages import HumanMessage

input = {"messages": [HumanMessage(content="what is happening with tesla today?")]}

# for output in app.stream(input):

#     for key, value in output.items():
#         print(f"Output from node '{key}':")
#         print("---")
#         print(value)
#     print("\n---\n")

output = app.invoke(input)
output

{'messages': [HumanMessage(content='what is happening with tesla today?'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "Tesla news today"\n}', 'name': 'tavily_search_results_json'}}),
  FunctionMessage(content='[{\'url\': \'https://www.fool.com/investing/2024/01/26/tesla-stock-plunged-52-rejoin-1-trillion-club-2024/\', \'content\': "Tesla\'s Q4 2023 financial results and forward guidance missed expectations  Tesla also spent most of 2023 slashing prices. By December, the average price of a new Tesla was down a whopping 25.1%  The reductions have continued in 2024, with Tesla cutting prices in China and Europe already.  Tesla sold 1.2 million Model Y EVs during 2023, making it the best-selling car of the year in all categories. TeslaTesla (TSLA 1.16%) stock peaked at more than $400 in 2021, which valued the company north of $1 trillion. But it has since plunged 52% from that high point amid a series of economic headwinds like ..."}]', name='t